In [1]:
import mxnet as mx
import numpy as np
import mnist
import logging

In [2]:
train_x, train_y,test_x, test_y = mnist.readmnist('data/mnist')

In [3]:
epochs = 50
batch_size = 100

In [21]:
#Setting iterators

train_iter = mx.io.NDArrayIter(data = train_x, label = train_y, batch_size = batch_size,data_name = 'data', 
                               label_name = 'softmax_label',last_batch_handle = "discard", shuffle = True)
data = mx.sym.Variable("data")
fc1 = mx.sym.FullyConnected(data = data, name ="fc1", num_hidden = 64, flatten = 1)
relu1 = mx.sym.Activation(data = fc1, act_type = "relu", name = 'relu1')
fc2 = mx.sym.FullyConnected(data = relu1, name = "fc2", num_hidden = 10, flatten = 1)
out = mx.sym.SoftmaxOutput(data = fc2, name = "softmax")

mod = mx.mod.Module(out,context = mx.cpu())
print(mod.data_names)
print(mod.label_names)

mod.bind(data_shapes = train_iter.provide_data, label_shapes = train_iter.provide_label, for_training = True)
mod.init_params(initializer = mx.init.Xavier(magnitude = 1.0))
mod.init_optimizer(optimizer = "sgd", optimizer_params=(('learning_rate',0.01),), force_init = False)
mod.fit(train_data = train_iter, eval_metric = "accuracy", num_epoch = epochs)
mod.score(eval_data = train_iter, eval_metric = ["acc","mse"])

/home/ubuntu/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/module/base_module.py:488: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)


['data']
['softmax_label']


[('accuracy', 0.9000016666666667), ('mse', 0.0064350389576672265)]

In [ ]:
test_iter = mx.io.NDArrayIter(data = test_x, labels = test_y, batch_size = batch_size, shuffle = False,
                              last_batch_handle = "discard")